In [ ]:
!pip install xarray

In [1]:
import os

os.environ['MAX_FLOWS_BUCKET'] = "hydrovis-ti-fim-us-east-1"
os.environ['CACHE_DAYS'] = "30"
os.environ['INITIALIZE_PIPELINE_FUNCTION'] = "test"

In [8]:
import json
import os
from datetime import datetime, timedelta
import xarray
import pandas as pd
import numpy as np
import boto3
import botocore
import urllib.parse
import time
import requests

from helper_functions.old_shared_funcs import get_configuration, check_s3_file_existence

MAX_FLOWS_BUCKET = os.environ['MAX_FLOWS_BUCKET']
CACHE_DAYS = os.environ['CACHE_DAYS']
INITIALIZE_PIPELINE_FUNCTION = os.environ['INITIALIZE_PIPELINE_FUNCTION']


def lambda_handler(event, context):
    """
        The lambda handler is the function that is kicked off with the lambda. This function will take all the
        forecast steps in the NWM configuration, calculate the max streamflow for each feature and then save the
        output in S3

        Args:
            event(event object): An event is a JSON-formatted document that contains data for a Lambda function to
                                 process
            context(object): Provides methods and properties that provide information about the invocation, function,
                             and runtime environment
    """
    # parse the event to get the bucket and file that kicked off the lambda
    print("Parsing key to get configuration")
    if "Records" in event:
        message = json.loads(event["Records"][0]['Sns']['Message'])
        data_key = urllib.parse.unquote_plus(message["Records"][0]['s3']['object']['key'], encoding='utf-8')
        data_bucket = message["Records"][0]['s3']['bucket']['name']
    else:
        data_key = event['data_key']
        data_bucket = event['data_bucket']

    if not check_s3_file_existence(data_bucket, data_key):
        raise Exception(f"{data_key} does not exist within {data_bucket}")

    max_flow_calcs = []  # Store max flows in list just in case multiplens needed
    metadata = get_configuration(data_key, mrf_timestep=3)
    configuration = metadata.get('configuration')
    date = metadata.get('date')
    hour = metadata.get('hour')
    forecast_timestep = metadata.get('forecast_timestep')

    # Setup an input files dictionary with the bucket
    input_files = metadata.get('input_files')
    files = []
    s3_file_sets = []
    for s3_key in input_files:
        files.append({'bucket': data_bucket, 's3_key': s3_key})
    s3_file_sets.append(files)

    if "medium_range" in configuration:
        configuration = "medium_range"

    short_hand_config = configuration.replace("analysis_assim", "ana").replace("short_range", "srf").replace("medium_range", "mrf")  # noqa
    short_hand_config = short_hand_config.replace("hawaii", "hi").replace("puertorico", "prvi")
    subset_config = None

    if configuration == 'medium_range':
        subset_config = f"{int(int(forecast_timestep)/24)}day"
        short_hand_config = f"{short_hand_config}_{subset_config}"

    max_flow_calcs.append(short_hand_config)  # Add the current configuration to calcs list.

    if configuration == 'analysis_assim':
        # Only perform max flows computation for analysis_assim when its the 00Z timestep
        if hour != "00":
            print(f"{data_key} is not a file for 00Z so max flows will not be calculated")
            return
        else:
            max_flow_calcs = []  # If running ana, reset the calcs list and s3_file_sets list
            s3_file_sets = []
            days = [14]
            for day in days:
                files = []
                previous_forecasts = day*24
                metadata = get_configuration(data_key, previous_forecasts=previous_forecasts)
                input_files = metadata.get('input_files')
                print(len(input_files))
                for s3_key in input_files:
                    files.append({'bucket': data_bucket, 's3_key': s3_key})
                s3_file_sets.append(files)
                reference_time = metadata.get('reference_time')
                subset_config = f"{int(previous_forecasts/24)}day"
                short_hand_config = f"ana_{subset_config}"
                max_flow_calcs.append(short_hand_config)

    print(f"Creating max flows file for {configuration} for {date}T{hour}:00:00Z")

    for n, max_flow_calc in enumerate(max_flow_calcs):
        output_netcdf = f"max_flows/{configuration}/{date}/{max_flow_calc}_{hour}_max_flows.nc"

        # Once the files exist, calculate the max flows
        calculate_max_flows(s3_file_sets[n], output_netcdf)
        print(f"Successfully created max flows file for {configuration} for {date}T{hour}:00:00Z")

        # If max calcs will run more than once, remove duplicates and
        # update the second set of input files to include the just created max file.
        next = n+1
        if len(s3_file_sets) > (next):
            s3_file_sets[next] = [file_next for file_next in s3_file_sets[next] if file_next not in s3_file_sets[n]]
            s3_file_sets[next].append({'bucket': MAX_FLOWS_BUCKET, 's3_key': output_netcdf})

    print("Done")


def calculate_max_flows(s3_files, output_netcdf):
    """
        Iterates through a times series of National Water Model (NWM) channel_rt output NetCDF files, and finds the
        maximum flow of each NWM reach during this period.  Outputs a NetCDF file containing all NWM reaches and their
        maximum flows.

        Args:
            data_bucket (str): S3 bucket name where the NWM files are stored
            path_to_nwm_files (str or list): Path to the directory or list of the paths to the files to caclulate
                                             maximum flows on.
            output_netcdf (str): Key (path) of the max flows netcdf that will be store in S3
    """
    print("--> Calculating flows")
    peak_flows, feature_ids = calc_max_flows(s3_files)  # creates a max flow array for all reaches

    print(f"--> Creating {output_netcdf}")
    write_netcdf(feature_ids, peak_flows, output_netcdf)  # creates the output NetCDF file


def download_file(data_bucket, file_path, download_path):
    s3 = boto3.client('s3')

    try:
        s3.download_file(data_bucket, file_path, download_path)
        return True
    except Exception as e:
        print(f"File failed to download {file_path}: {e}")
        return False


def calc_max_flows(s3_files):
    """
        Iterates through a times series of National Water Model (NWM) channel_rt output NetCDF files, and finds
        the maximum flow of each NWM reach during this period.

        Args:
            data_bucket (str): S3 bucket name where the NWM files are stored
            active_file_paths (str or list): Path to the directory or list of the paths to the files to caclulate
                                             maximum flows on.

        Returns:
            max_flows (numpy array): Numpy array that contains all the max flows for each feature for the forecast
            feature_ids (numpy array): Numpy array that contains all the features ids for the forecast
    """
    max_flows = None
    feature_ids = None
    tries = 0

    for file in s3_files:
        data_bucket = file['bucket']
        file_path = file['s3_key']
        download_path = f'/tmp/{os.path.basename(file_path)}'
        
        if check_s3_file_existence(data_bucket, file_path):
            print(f"--> Downloading {file_path} to {download_path}")
            download_file(data_bucket, file_path, download_path)
        else:
            file_path = file_path.replace('common/data/model/com/nwm/prod', 'https://storage.googleapis.com/national-water-model')
            print(f"--> Downloading {file_path} to {download_path}")
            open(download_path, 'wb').write(requests.get(file_path, allow_redirects=True).content)

        with xarray.open_dataset(download_path) as ds:
            temp_flows = ds['streamflow'].values  # imports the streamflow values from each file
            if max_flows is None:
                max_flows = temp_flows
            if feature_ids is None:
                feature_ids = ds['feature_id'].values

        print(f"--> Removing {download_path}")
        os.remove(download_path)

        # compares the streamflow values in each file with those stored in the max_flows array, and keeps the
        # maximum value for each reach
        max_flows = np.maximum(max_flows, temp_flows)

    return max_flows, feature_ids

def write_netcdf(feature_ids, peak_flows, output_netcdf):
    """
        Iterates through a times series of National Water Model (NWM) channel_rt output NetCDF files, and finds the
        maximum flow of each NWM reach during this period.

        Args:
            feature_ids (numpy array): Numpy array that contains all the features ids for the forecast
            peak_flows (numpy array): Numpy array that contains all the max flows for each feature for the forecast
            output_netcdf (str or list): Key (path) of the max flows netcdf that will be store in S3
    """
    s3 = boto3.client('s3')

    tmp_netcdf = '/tmp/max_flows.nc'

    # Create a dataframe from the feature ids and streamflow
    df = pd.DataFrame(feature_ids, columns=['feature_id']).set_index('feature_id')
    df['streamflow'] = peak_flows
    df['streamflow'] = df['streamflow'].fillna(0)

    # Save the max flows dataframe to a loacl netcdf file
    df.to_xarray().to_netcdf(tmp_netcdf)

    # Upload the local max flows file to the S3 bucket
    s3.upload_file(tmp_netcdf, MAX_FLOWS_BUCKET, output_netcdf, ExtraArgs={'ServerSideEncryption': 'aws:kms'})
    os.remove(tmp_netcdf)

In [9]:
event = {
  "data_bucket": "hydrovis-ti-nwm-us-east-1",
  "data_key": "common/data/model/com/nwm/prod/nwm.20221007/analysis_assim/nwm.t00z.analysis_assim.channel_rt.tm00.conus.nc"
}

lambda_handler(event, {})

Parsing key to get configuration
337
Creating max flows file for analysis_assim for 20221007T00:00:00Z
--> Calculating flows


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


--> Downloading https://storage.googleapis.com/national-water-model/nwm.20220923/analysis_assim/nwm.t00z.analysis_assim.channel_rt.tm00.conus.nc to /tmp/nwm.t00z.analysis_assim.channel_rt.tm00.conus.nc
--> Removing /tmp/nwm.t00z.analysis_assim.channel_rt.tm00.conus.nc
--> Downloading https://storage.googleapis.com/national-water-model/nwm.20220923/analysis_assim/nwm.t01z.analysis_assim.channel_rt.tm00.conus.nc to /tmp/nwm.t01z.analysis_assim.channel_rt.tm00.conus.nc


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


--> Removing /tmp/nwm.t01z.analysis_assim.channel_rt.tm00.conus.nc
--> Downloading https://storage.googleapis.com/national-water-model/nwm.20220923/analysis_assim/nwm.t02z.analysis_assim.channel_rt.tm00.conus.nc to /tmp/nwm.t02z.analysis_assim.channel_rt.tm00.conus.nc


...

--> Removing /tmp/nwm.t23z.analysis_assim.channel_rt.tm00.conus.nc
--> Downloading common/data/model/com/nwm/prod/nwm.20221007/analysis_assim/nwm.t00z.analysis_assim.channel_rt.tm00.conus.nc to /tmp/nwm.t00z.analysis_assim.channel_rt.tm00.conus.nc


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


--> Removing /tmp/nwm.t00z.analysis_assim.channel_rt.tm00.conus.nc
--> Creating max_flows/analysis_assim/20221007/ana_14day_00_max_flows.nc


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Successfully created max flows file for analysis_assim for 20221007T00:00:00Z
Done
